In [3]:
# Tested on
# Python 3.5.3 :: Anaconda 4.4.0 (64-bit)
# scipy 0.19.1
# keras 2.0.6 <- not required to reproduce the result of Fig. 1-2 of our paper.
# tensorflow-gpu 1.2.1 <- not required to reproduce the result of Fig. 1-2 of our paper.
# numpy 1.13.1
# pylab using matplotlib backend: Qt5Agg

In [1]:
pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [5]:
## STEP 1-1. 
## Define scripts


## Preprocessing ##
from tqdm import tqdm
import numpy as np
import csv

def smoothing(dat_, bin_=16):
    # Calculate F/DeltaF from signal
    buffer = np.zeros(len(dat_) - bin_)
    for num in range(len(dat_)-bin_):
        buffer[num] = (dat_[num+bin_] / dat_[num:num+bin_].mean())-1
    return (buffer * (buffer>0) * (buffer<2))/2 + (2 * (buffer>=2))

def preprocessing(path):
    # Calculate 255 * F_norm for converting 8 bit integer
    with open(path, newline='') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='"')
        labels = next(spamreader)
        data_original = np.array([row for row in spamreader], dtype=float).T
        data_smoothed = np.array([255*smoothing(elem) for elem in data_original])
    data_normalized = np.array([255*(elem-min(elem))/(max(elem)-min(elem)) for elem in data_smoothed])
    # Reduce dimension from 18 to 9
    data_preprocessed = []
    for num in range(9):
        data_preprocessed.append(np.maximum(data_normalized[num], data_normalized[17-num]))
    return np.asarray(data_preprocessed)

In [6]:
## STEP 1-2
## Import data and preprocessing

data_pre = []
data_path = ['/csv/No01.csv',
             '/csv/No02.csv',
             '/csv/No03.csv',
             '/csv/No04.csv',
             '/csv/No05.csv',
             '/csv/No06.csv',
             '/csv/No07.csv',
             '/csv/No08.csv',
             '/csv/No09.csv',
             '/csv/No10.csv',
             '/csv/No11.csv',
             '/csv/No12.csv']
for path in tqdm(data_path):
    data_pre.append(preprocessing(path))

100%|██████████| 12/12 [00:07<00:00,  1.68it/s]


In [7]:
## STEP 2-1
## Go to STEP 2-3 if you cannot use Keras.

## Import Keras (important note : use TensorFlow backend)
from keras.applications.vgg16 import VGG16
from keras.models import Model

## Import Imagnet pre-trained VGG16 ##
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(72,72,3))
model = Model(inputs=base_model.input, outputs=base_model.get_layer('block4_conv3').output)

## Feature Extraction from RGB images##
from tqdm import tqdm
import cv2
def feature_extraction(data, window_size):
    buffer = []
    data_len = len(data)
    for num in tqdm(range(len(data[0])-window_size)):
        image_pre = np.uint8(data[:,num:num+window_size])
        image_channel = cv2.merge((image_pre,image_pre,image_pre))
        image_input = cv2.resize(image_channel, (0,0), fx=(72/window_size), fy=(72/data_len))
        Conv4_3 = model.predict(np.expand_dims(image_input, axis=0))
        GAP = Conv4_3.mean(axis=1).mean(axis=1)[0]
        buffer.append(GAP)
    return np.asarray(buffer)

In [8]:
## STEP 2-2
## Extract global average pooled value (GAP) from VGG16 Conv4_3

vgg = np.zeros((0,512), dtype=float32)
for ind in range(len(data_pre)):
    vgg = np.append(vgg, feature_extraction(data_pre[ind], 8), axis=0)
    
# calculation time < 30 min.
# If you want to shorten the calculation time, use multiprocessing.

100%|██████████| 2024/2024 [02:10<00:00, 15.55it/s]


In [23]:
## STEP 2-3
## Import calculated data which used in our paper.
import pickle
with (open('/pickle/Fig1vgg16_00.pickle', "rb" )) as openfile:
    vgg_used_00 = pickle.load(openfile)
with (open('/pickle/Fig1vgg16_01.pickle', "rb" )) as openfile:
    vgg_used_01 = pickle.load(openfile)
vgg_used = np.append(vgg_used_00, vgg_used_01, axis=0)
## If you skipped STEP 2-1 and 2-2, you can use vgg_used as vgg (vgg = vgg_used.copy())

In [24]:
## STEP 2-4 (Check)

## Check datatype
print('Same datatype =', vgg_used.dtype == vgg.dtype)

## Checksum
print('Checksum =', abs((vgg_used - vgg)).sum() == 0)

## The checksum must be 0.0
## If they are not true, clustering result may differ.

Same datatype = True
Checksum = True


In [11]:
## STEP 3-1
## Hierarchical Clustering using Ward's method

from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import dendrogram, linkage, cophenet, fcluster

Z = linkage(vgg, 'ward')
c, coph_dists = cophenet(Z, pdist(vgg))

# Calculation time < 10 min.

In [12]:
## STEP 3-2
## Get cluster label 

max_d = 5000
cluster = fcluster(Z, max_d, criterion='distance')

#Use max_d = 5000 to reproduce our result.
#Adjust max_d if you need. 

In [13]:
## STEP 3-3
## Match the cluster label and signal data

buffer = np.zeros(0)
data_tot = len(data_pre)
window_size = 8
data_len = len(data_pre[0][0]) - window_size

for num in range(data_tot):
    buffer = np.append(buffer, np.zeros((window_size//2)))
    buffer = np.append(buffer, cluster[data_len*num:data_len*(num+1)])
    buffer = np.append(buffer, np.zeros((window_size//2 + window_size%2)))
clusters_comp = buffer.copy()

buffer = np.zeros((9,0))
for ind in range(data_tot):
    buffer = np.append(buffer, data_pre[ind], axis=1)
data_pre_comp = buffer.copy()

buffer = np.zeros(0)
for num in range(data_tot):
    buffer = np.append(buffer, cluster[data_len*num:data_len*(num+1)])
    buffer = np.append(buffer, np.zeros(window_size))
clusters_matched = buffer.copy()


In [14]:
## STEP 3-4
## post-hoc human labeling using average image of cluster

def cluster_mean_image(dat, num_clust, window_size=8):
    temp = []
    for num in range(len(dat)):
        if dat[num] == num_clust:
            temp.append(data_pre_comp[:, num:num+window_size])
    return np.mean(np.asarray(temp), axis=0)

f, axarr = plt.subplots(1,int(max(clusters_matched)), sharex=True, frameon=False)
for num in range(int(max(clusters_matched))):
    axarr[num].imshow(cluster_mean_image(clusters_matched, num+1)/255, aspect='auto', vmin=0, vmax=.5, cmap='Greys_r')
    axarr[num].set_xlabel(num+1)
    axarr[num].set_yticklabels([])
    axarr[num].set_xticklabels([])

In [15]:
## STEP 3-5
## Get activity data and final check

AT = np.zeros(len(clusters_matched))
BW = np.zeros(len(clusters_matched))
FW = np.zeros(len(clusters_matched))
PT = np.zeros(len(clusters_matched))
QS = np.zeros(len(clusters_matched))

for num in [19,20,21,22,23]:
    AT += (clusters_comp == num)
for num in [4,5,6]:
    BW += (clusters_comp == num)
for num in [15,16]:
    FW += (clusters_comp == num)
for num in [11,12,13]:
    PT += (clusters_comp == num)
for num in [1]:
    QS += (clusters_comp == num)

f, axarr = plt.subplots(2, sharex=True)
axarr[0].imshow(data_pre_comp/255, cmap='Greys_r', vmin=0, vmax=1, aspect='auto')
axarr[1].plot(1.0 * AT, c='purple')
axarr[1].plot(0.8 * BW, c='red')
axarr[1].plot(0.6 * FW, c='blue')
axarr[1].plot(0.4 * PT, c='green')
axarr[1].plot(0.2 * QS, c='grey')

In [17]:
## STEP 3-6 (Check)
## Compare labeled activity and data which we used in Fig. 1-2 of our paper.

import pickle
with (open('/pickle/Fig1acts.pickle', "rb" )) as openfile:
    acts_used = pickle.load(openfile)

acts = [AT, BW, FW, PT, QS]
for ind in range(len(acts)):
    print(abs(acts[ind]-acts_used[ind]).sum() == 0)
    
## print True 5 times

True
True
True
True
True


In [18]:
## GROUND TRUTH

import pickle
with (open('/pickle/Fig1groundtruth.pickle', "rb" )) as openfile:
    ground_truth = pickle.load(openfile)

#the value of ground_truth data
#1 = AT, 0.8 = BW, 0.6 = FW, 0.4 = PT
# [0:400] of ground_truth = [816:1216] of sample 1
# [400:800] of ground_truth = [816:1216] of sample 2, and so on


In [19]:
## Check GROUND TRUTH using sample 1.

f, axarr = plt.subplots(3, sharex=True)
axarr[0].imshow(data_pre_comp[:,816:1216]/255, cmap='Greys_r', vmin=0, vmax=1, aspect='auto')
axarr[1].plot(1.0 * AT[816:1216], c='purple')
axarr[1].plot(0.8 * BW[816:1216], c='red')
axarr[1].plot(0.6 * FW[816:1216], c='blue')
axarr[1].plot(0.4 * PT[816:1216], c='green')
axarr[1].plot(0.2 * QS[816:1216], c='grey')
axarr[2].plot(1.0 * (ground_truth==1)[0:400], c='purple')
axarr[2].plot(0.8 * (ground_truth==.8)[0:400], c='red')
axarr[2].plot(0.6 * (ground_truth==.6)[0:400], c='blue')
axarr[2].plot(0.4 * (ground_truth==.4)[0:400], c='green')